In [2]:
from txgnn import TxData, TxGNN, TxEval

TxData = TxData(data_folder_path = 'data/train_data')
TxData.prepare_split(split = 'complex_disease', seed = 42, no_kg = False)

Using backend: pytorch


Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!


In [3]:
TxGNN = TxGNN(data = TxData, 
              weight_bias_track = False,
              proj_name = 'TxGNN',
              exp_name = 'TxGNN'
              )

# to load a pretrained model: 
TxGNN.load_pretrained('./models/checkpoints_all_seeds/TxGNN_1_complex_disease')

# TxGNN.model_initialize(n_hid = 100, 
#                       n_inp = 100, 
#                       n_out = 100, 
#                       proto = True,
#                       proto_num = 3,
#                       attention = False,
#                       sim_measure = 'all_nodes_profile',
#                       bert_measure = 'disease_name',
#                       agg_measure = 'rarity',
#                       num_walks = 200,
#                       walk_mode = 'bit',
#                       path_length = 2)

In [3]:
## here we did not run this, since the output is too long to fit into the notebook
TxGNN.pretrain(n_epoch = 2, 
               learning_rate = 1e-3,
               batch_size = 1024, 
               train_print_per_n = 20)

Creating minibatch pretraining dataloader...
Start pre-training with #param: 1015000
Epoch: 0 Step: 0 LR: 0.00100 Loss 0.6933, Pretrain Micro AUROC 0.4993 Pretrain Micro AUPRC 0.5083 Pretrain Macro AUROC 0.5110 Pretrain Macro AUPRC 0.6172
Epoch: 0 Step: 20 LR: 0.00100 Loss 0.6781, Pretrain Micro AUROC 0.6009 Pretrain Micro AUPRC 0.5960 Pretrain Macro AUROC 0.5835 Pretrain Macro AUPRC 0.6718
Epoch: 0 Step: 40 LR: 0.00100 Loss 0.6662, Pretrain Micro AUROC 0.6323 Pretrain Micro AUPRC 0.6109 Pretrain Macro AUROC 0.6269 Pretrain Macro AUPRC 0.6895
Epoch: 0 Step: 60 LR: 0.00100 Loss 0.6669, Pretrain Micro AUROC 0.6166 Pretrain Micro AUPRC 0.5919 Pretrain Macro AUROC 0.5775 Pretrain Macro AUPRC 0.6453
Epoch: 0 Step: 80 LR: 0.00100 Loss 0.6520, Pretrain Micro AUROC 0.6463 Pretrain Micro AUPRC 0.6236 Pretrain Macro AUROC 0.5831 Pretrain Macro AUPRC 0.6962
Epoch: 0 Step: 100 LR: 0.00100 Loss 0.6608, Pretrain Micro AUROC 0.6367 Pretrain Micro AUPRC 0.6018 Pretrain Macro AUROC 0.6586 Pretrain Macr

KeyboardInterrupt: 

In [5]:
## here as a demo, the n_epoch is set to 30. Change it to n_epoch = 500 when you use it
TxGNN.finetune(n_epoch = 30, 
               learning_rate = 5e-4,
               train_print_per_n = 5,
               valid_per_n = 20)

Epoch: 0 LR: 0.00050 Loss 0.6938, Train Micro AUROC 0.5109 Train Micro AUPRC 0.5127 Train Macro AUROC 0.5019 Train Macro AUPRC 0.5073
----- AUROC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.5349347764149285
('drug', 'indication', 'disease'): 0.5475017777377161
('drug', 'off-label use', 'disease'): 0.4750877230935641
('disease', 'rev_contraindication', 'drug'): 0.5200441594466088
('disease', 'rev_indication', 'drug'): 0.42414272145771154
('disease', 'rev_off-label use', 'drug'): 0.5098947539210383
----- AUPRC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.5439511651288017
('drug', 'indication', 'disease'): 0.551160435487897
('drug', 'off-label use', 'disease'): 0.4725924166335391
('disease', 'rev_contraindication', 'drug'): 0.5207788369454536
('disease', 'rev_indication', 'drug'): 0.4539515055664601
('disease', 'rev_off-label use', 'drug'): 0.5012481020151309
----------------------------------------------
Validation.....
E

In [8]:
#TxGNN.save_model('./models/checkpoints_all_seeds/TxGNN_2_complex_disease')
TxGNN.load_pretrained('/home/apakiman/Repo/merck_gds_explr/submodules/TxGNN/models/checkpoints_all_seeds/TxGNN_1_complex_disease')

In [4]:
TxGNN.train_graphmask(relation = 'indication',
                      learning_rate = 3e-4,
                      allowance = 0.005,
                      epochs_per_layer = 3,
                      penalty_scaling = 1,
                      valid_per_n = 20)

output = TxGNN.retrieve_save_gates('./model_ckpt')
TxGNN.save_graphmask_model('./graphmask_model_ckpt')

gate_hidden_size:  32
Enabling layer 1
Running epoch 0 of GraphMask training. Mean divergence=0.0000, mean penalty=1.8973, bce_update=0.3006, bce_original=0.3006, num_masked_l1=0.0000, num_masked_l2=0.0000
----- validation Result -----
Epoch 0, Mean divergence=0.0000, mean penalty=1.8954, bce_update=0.6552, bce_original=0.6552, num_masked_l1=0.0000, num_masked_l2=0.0000
-------------------------------


: 

In [13]:
from txgnn import TxEval
TxEval = TxEval(model = TxGNN)

load_pretrained_graphmask()

In [15]:
# evaluate individual diseases
result = TxEval.eval_disease_centric(disease_idxs = [12661.0, 11318.0], 
                                     relation = 'indication', 
                                     save_result = False)

# evaluate the entire test set
result = TxEval.eval_disease_centric(disease_idxs = 'test_set', 
                                     show_plot = False, 
                                     verbose = True, 
                                     save_result = True,
                                     return_raw = False)

  0%|          | 0/2 [00:00<?, ?it/s]

Evaluating relation: contraindication


  0%|          | 0/54 [00:00<?, ?it/s]

---------
AUROC mean:  0.752327526138064
AUROC std:  0.1858856686489882
---------
---------
AUPRC mean:  0.13968016130970864
AUPRC std:  0.2302530155722306
---------
---------
Accuracy mean:  0.8823494912172664
Accuracy std:  0.0646862634452154
---------
---------
Sensitivity mean:  0.4061223240656242
Sensitivity std:  0.39641674378976877
---------
---------
Specificity mean:  0.896884633314133
Specificity std:  0.06615214214692447
---------
---------
F1 mean:  0.11826961006777822
F1 std:  0.1974920901665274
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9817090124506684
NPV std:  0.03551809172552505
---------
---------
FPR mean:  0.10311536668586684
FPR std:  0.06615214214692448
---------
---------
FNR mean:  0.5938776759343759
FNR std:  0.39641674378976877
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  32.01851851851852
# of Pos std:  55.05165612631155
---------
---------
Recall@1% mean:  0.12043450669812626
Rec

  0%|          | 0/66 [00:00<?, ?it/s]

---------
AUROC mean:  0.8257557356962596
AUROC std:  0.16985070951055026
---------
---------
AUPRC mean:  0.07869411551085655
AUPRC std:  0.12907962935380474
---------
---------
Accuracy mean:  0.9232244712533441
Accuracy std:  0.045589373587365895
---------
---------
Sensitivity mean:  0.4268625714941528
Sensitivity std:  0.42846127191825706
---------
---------
Specificity mean:  0.9253084621815284
Specificity std:  0.04604332812527355
---------
---------
F1 mean:  0.0390161088191038
F1 std:  0.0557114334640408
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9974256498725271
NPV std:  0.005621376482176378
---------
---------
FPR mean:  0.07469153781847164
FPR std:  0.04604332812527355
---------
---------
FNR mean:  0.5731374285058471
FNR std:  0.42846127191825706
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  7.5
# of Pos std:  12.41608194677737
---------
---------
Recall@1% mean:  0.15907967319709976
Recall@1% s

  0%|          | 0/35 [00:00<?, ?it/s]

---------
AUROC mean:  0.8403037698819223
AUROC std:  0.1511260204653165
---------
---------
AUPRC mean:  0.13293952998553243
AUPRC std:  0.2118786815594834
---------
---------
Accuracy mean:  0.907187222715173
Accuracy std:  0.066338464223857
---------
---------
Sensitivity mean:  0.49357142857142855
Sensitivity std:  0.450846143266147
---------
---------
Specificity mean:  0.9095522375161197
Specificity std:  0.06736574584064527
---------
---------
F1 mean:  0.07526814486574862
F1 std:  0.12204437231852404
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9967035858737601
NPV std:  0.004315405206175934
---------
---------
FPR mean:  0.09044776248388027
FPR std:  0.06736574584064525
---------
---------
FNR mean:  0.5064285714285715
FNR std:  0.450846143266147
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  3.3142857142857145
# of Pos std:  2.4351172288757157
---------
---------
Recall@1% mean:  0.08
Recall@1% std:  0

In [16]:
TxEval.retrieve_disease_idxs_test_set('indication')

array([ 7701., 12661., 11195., 16569.,  6343.,  1479., 12621., 14416.,
       13557., 16501., 13446., 13610.,  7105.,  9716.,  5822., 12320.,
       10023., 14513., 12929.,  5683.,  9950., 14661.,  8395.,  7990.,
        6031.,  6255., 14409.,  9589., 11885.,  4285.,  6377., 13130.,
        7773.,  9741., 13381., 15355.,  4536., 12891.,  9087., 14873.,
        6185.,  1812., 15301.,  6474.,  3381.,  6599., 16965.,  7880.,
       16698.,  6252.,  5764.,  4192.,  4467., 13107., 14501., 11318.,
        3452., 11349., 15911., 15100.,  1417.,  4152.,  1319.,  2220.,
       15989.,  8973.])